In [1]:
from dockerComposeGenerator import dockerComposeGenerator as g

### Enter your service commands between the triple quotes. Press <Ctrl + Enter> to run the selected cell.

In [2]:
txt = g('''
docker service create \
--name voca_redis \
--constraint 'node.role == manager' \
--replicas=1 \
--network voca_network \
--hostname voca_redis \
redis:3.2.7
docker service create \
--name airflow-webserver \
--publish 2048:4040 \
--publish 30000:8080 \
--replicas=1 \
--network voca_network \
--constraint 'node.role == manager' \
artifactory.associatesys.local:6555/aa/airflow:test-0.0.5 webserver 
docker service create \
--name airflow-workers \
--mount type=volume,source=my-volume,destination=/path/in/container \
--publish 8793:8793 \
--replicas=4 \
--network voca_network \
--constraint 'node.role != manager' \
artifactory.associatesys.local:6555/aa/airflow:test-0.0.5 worker
''')



version: "3"
services:
    voca_redis:
        image: redis:3.2.7
        deploy:
            replicas: 1
            placement:
                constraints: [node.role == manager]
        networks:
            voca_network:
                 aliases:
                     - voca_redis
    airflow-webserver:
        image: artifactory.associatesys.local:6555/aa/airflow:test-0.0.5
        command: webserver
        deploy:
            replicas: 1
            placement:
                constraints: [node.role == manager]
        ports:
            - "2048:4040"
            - "30000:8080"
        networks:
            - voca_network
    airflow-workers:
        image: artifactory.associatesys.local:6555/aa/airflow:test-0.0.5
        command: worker
        deploy:
            replicas: 4
            placement:
                constraints: [node.role != manager]
        volumes:
            - type: volume
              source: my-volume
              target: /path/in/container
        port

In [3]:
help(g)


Help on function dockerComposeGenerator in module composeGenerator:

dockerComposeGenerator(cmd)
    Takes docker service create command and generates an equivelent docker compose file
    Surround your multi word constraints with single quotes.
    Command structure is "docker service create [OPTIONS] IMAGE [COMMAND]"

